In [4]:
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn as nn

# Ma trận liền kề A (giả định 3 vùng để minh họa)
A = np.array([
    [1, 1, 1],  # Region 1 liền kề 1, 2, 3
    [1, 1, 0],  # Region 2 liền kề 1, 2
    [1, 0, 1]   # Region 3 liền kề 1, 3
])

# Đặc trưng nút (node features) - từ dữ liệu của bạn và giả định thêm
features = np.array([
    [20804.0, 0.6412, 0.3398, 0.7225, 7992.0],  # Region 1
    [15000.0, 0.7000, 0.4500, 0.8000, 5000.0],  # Region 2 (giả định)
    [18000.0, 0.6500, 0.4000, 0.7500, 6000.0]   # Region 3 (giả định)
])

# LPIPS (edge features) - từ dữ liệu của bạn và giả định thêm
lpips_data = np.array([
    [1, 2, 0.143],  # Region 1 và 2
    [1, 3, 0.200],  # Region 1 và 3 (giả định)
    [2, 1, 0.143]   # Region 2 và 1 (đối xứng)
])

# Ground truth Y = A
edge_index = np.where(A > 0)
edge_index = torch.tensor(edge_index, dtype=torch.long)
Y = torch.tensor(A[edge_index[0], edge_index[1]], dtype=torch.float)

# Tạo edge_attr từ LPIPS
edge_attr = []
for i, j in zip(edge_index[0].numpy(), edge_index[1].numpy()):
    pair = lpips_data[(lpips_data[:, 0] == i+1) & (lpips_data[:, 1] == j+1)]
    if len(pair) > 0:
        edge_attr.append(pair[0, 2])
    else:
        edge_attr.append(0.5)  # Giá trị mặc định nếu thiếu
edge_attr = torch.tensor(edge_attr, dtype=torch.float)

# Tạo graph data
data = Data(
    x=torch.tensor(features, dtype=torch.float),  # Node features
    edge_index=edge_index,                        # Edges
    edge_attr=edge_attr,                          # Edge features (LPIPS)
    y=Y                                           # Ground truth
)

In [5]:
class ContourMerger(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers):
        super(ContourMerger, self).__init__()
        self.convs = nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
        self.convs.append(GCNConv(hidden_channels, out_channels))
        self.mlp = nn.Sequential(
            nn.Linear(out_channels + 1, 64),  # +1 cho edge_attr (LPIPS)
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x, edge_index, edge_attr):
        # GCN layers
        for conv in self.convs[:-1]:
            x = conv(x, edge_index)
            x = torch.relu(x)
        x = self.convs[-1](x, edge_index)  # H^{(L)}
        
        # Tạo edge features: trung bình đặc trưng nút + LPIPS
        E = (x[edge_index[0]] + x[edge_index[1]]) / 2
        E = torch.cat([E, edge_attr.unsqueeze(1)], dim=1)  # Kết hợp với LPIPS
        
        # MLP dự đoán Y_hat
        Y_hat = self.mlp(E)
        return Y_hat

# Khởi tạo mô hình
model = ContourMerger(
    in_channels=features.shape[1],  # 5 (Area, Extent, Aspect Ratio, Solidity, Deviation)
    hidden_channels=64,
    out_channels=32,
    num_layers=3
)

In [6]:
# Định nghĩa optimizer và loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()
# Huấn luyện
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    # Dự đoán
    Y_hat = model(data.x, data.edge_index, data.edge_attr)
    # Tính loss
    loss = criterion(Y_hat.squeeze(), data.y)
    # Backpropagation
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")
model.eval()
with torch.no_grad():
    Y_hat = model(data.x, data.edge_index, data.edge_attr)
    predictions = (Y_hat > 0.5).float()
    accuracy = (predictions.squeeze() == data.y).float().mean()
    print(f"Accuracy: {accuracy.item():.4f}")

Epoch 0, Loss: 0.0000
Epoch 10, Loss: 0.0000
Epoch 20, Loss: 0.0000
Epoch 30, Loss: 0.0000
Epoch 40, Loss: 0.0000
Epoch 50, Loss: 0.0000
Epoch 60, Loss: 0.0000
Epoch 70, Loss: 0.0000
Epoch 80, Loss: 0.0000
Epoch 90, Loss: 0.0000
Accuracy: 1.0000
